In [2]:
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import re
import os

def calculate_magnitude(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

def cleaning101(data):
    df = data.copy()
    x = np.array(df['x'].values)
    y = np.array(df['y'].values)
    z = np.array(df['z'].values)
    magnitude = calculate_magnitude(x, y, z)
    df['magnitude'] = magnitude

    fs = 50  # Sampling frequency
    lowcut = 0.4  # Lower cutoff frequency (Hz)
    highcut = 15  # Upper cutoff frequency (Hz)
    signal = np.array(df['magnitude'])

    filtered_data = bandpass_filter(signal, lowcut, highcut, fs)
    df['filtered_magnitude'] = filtered_data
    # df.to_csv(filename, index=False)
    return df

def segment_and_flatten_magnitude(label, df, magnitude_column_name='filtered_magnitude', window_size_sec=5, overlap=0.5, sampling_rate=50):
    """
    1) Combine x, y, z signals into a single magnitude signal.
    2) Segment the magnitude signal into windows of window_size_sec seconds 
    with the specified overlap.
    3) Flatten each window into columns: start_time, x1, x2, ..., xN 
    (where N = window_size_sec * sampling_rate).
    
    Parameters:
        df (pd.DataFrame): DataFrame containing columns 'time', 'x', 'y', 'z'.
                        'time' is in milliseconds.
        magnitude_column_name (string): Target column name that already calculate magnitude.
        window_size_sec (float): Length of each window in seconds (default 5).
        overlap (float): Fraction of window overlap (default 0.5 = 50%).
        sampling_rate (int): Sampling rate in Hz (default 50).
        
    Returns:
        pd.DataFrame: Each row corresponds to a flattened window.
                    Columns:
                    - 'start_time': millisecond timestamp of the window start
                    - 'x1', 'x2', ..., 'xN': magnitude values for each sample
    """
    # Sort by time to ensure chronological order
    df = df.sort_values(by='time').reset_index(drop=True)

    # 2) Calculate number of samples per window and step size
    window_samples = int(window_size_sec * sampling_rate)  # e.g., 5s * 50Hz = 250
    step_size = int(window_samples * (1 - overlap))        # e.g., 250 * 0.5 = 125

    flattened_rows = []

    # 3) Loop through data with the given step size
    for start_idx in range(0, len(df) - window_samples + 1, step_size):
        # Extract this window of magnitude values
        window = df.iloc[start_idx : start_idx + window_samples].reset_index(drop=True)
        
        # Prepare a dict for one flattened row
        row_dict = {}
        row_dict['start_time'] = window.loc[0, 'time']
        row_dict['label'] = label
        
        # Flatten magnitude into x1, x2, x3, ..., xN
        for i in range(window_samples):
            row_dict[f'x{i+1}'] = window.loc[i, magnitude_column_name]
        
        flattened_rows.append(row_dict)

    # Convert list of dicts into a DataFrame
    return pd.DataFrame(flattened_rows)

def compute_fft_on_flattened_data(df, num_samples=250, remove_flatten=True):
    """
    Compute the FFT of the time-series in columns x1...xN, take the magnitude spectrum, 
    and keep only the first half (non-redundant part).
    
    The returned DataFrame contains:
    - 'start_time' and 'label'
    - Original data (x1-x250) if remove_flatten=False
    - FFT features starting from x251 (fft1-fft126)
    """
    feature_columns = [f'x{i+1}' for i in range(num_samples)]
    fft_features_list = []

    # Loop through each row to compute FFT
    for _, row in df.iterrows():
        ts = row[feature_columns].values.astype(np.float64)
        fft_vals = np.fft.fft(ts)
        fft_mag = np.abs(fft_vals)
        half_fft = fft_mag[:num_samples // 2 + 1]  # Keep first half (+1 for DC component)
        fft_features_list.append(half_fft)

    # Create FFT feature DataFrame
    num_fft_features = num_samples // 2 + 1
    fft_df = pd.DataFrame(
        fft_features_list, 
        columns=[f'x{i+1}' for i in range(num_samples+1, num_samples+1+num_fft_features)]
    )

    # Include original data if required
    if not remove_flatten:
        combined_df = pd.concat([df[['start_time', 'label'] + feature_columns].reset_index(drop=True), fft_df], axis=1)
    else:
        combined_df = pd.concat([df[['start_time', 'label']].reset_index(drop=True), fft_df], axis=1)

    return combined_df

In [ ]:
# Step 1 clean and flatten data
root_path = r"D:\git_repository\dsp_project\ignore_dir\datasets\cleaned_data_many_position"
# root_path = r"D:\git_repository\dsp_project\ignore_dir\datasets\clean_datasets"
magnitude_column_name = "filtered_magnitude"  # Adjust if needed
# magnitude_column_name = "magnitude"  # raw magnitude
window_size = 2  # Window size in seconds
overlap = 0.5

# Function to extract label from filename
def extract_label(filename):
    match = re.match(r'([a-zA-Z_]+)', filename)
    return match.group(1) if match else filename

# Process all CSV files in the directory and subdirectories
all_data = []
for dirpath, _, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith(".csv"):  # Process only CSV files
            file_path = os.path.join(dirpath, filename)
            label = extract_label(filename)
            data = pd.read_csv(file_path)
            # cleaned_data = data
            cleaned_data = cleaning101(data)
            processed_data = segment_and_flatten_magnitude(
                label=label, magnitude_column_name=magnitude_column_name, 
                df=cleaned_data, window_size_sec=window_size, overlap=overlap
            )
            all_data.append(processed_data)

# Combine all processed data
if all_data:
    merged_df = pd.concat(all_data, ignore_index=True)
    print("Merged DataFrame shape:", merged_df.shape)
else:
    print("No CSV files found.")


# compute flatten data with fft
fft_df = compute_fft_on_flattened_data(merged_df, num_samples=merged_df.shape[1]-2, remove_flatten=True) # -2 for start_time and label column on df

df = fft_df.copy()

# 1) Encode labels
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 2) Separate features and target
X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
y = df['label_encoded']

# 3) Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_test_raw = X_test
X_train_raw = X_train
# 4) Normalize features (0-1 scaling)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5) Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# 7) Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))

Merged DataFrame shape: (5966, 102)
                             precision    recall  f1-score   support

            climbing_stairs       0.84      0.88      0.86       161
          descending_stairs       0.93      0.75      0.83       149
                    nothing       0.99      0.93      0.96       253
                    running       1.00      1.00      1.00       213
sitting_standing_transition       0.91      0.98      0.94       208
                    walking       0.88      0.96      0.92       210

                   accuracy                           0.93      1194
                  macro avg       0.92      0.92      0.92      1194
               weighted avg       0.93      0.93      0.93      1194



In [74]:
# i working on digital signal processing project that use for regcognize human activity using accelerometer on android phone 
# like walking, running, cliambing stair and etc. as you can see my code work well with data that collect and convert into csv file. 
# Raw csv file have 3 columns is time in millis second x, y, z is raw value of accelerometer. My sampling rate is 50 Hz

# this is my old process data flow 
# Step 1 combine x,y,z with calculate_magnitude function 
# Step 2 pass calculated magnitude into band pass filter 
# Step 3 flatten with window size 2 (second) and overlab 50%
# Step 4 compute fft with flattened data and keep first half (+1 for DC component)
# Step 5 Training model
    # encode class name(string) into number with labelencoder
    # split train and test
    # Normalize features (0-1 scaling) using minmaxscaler
    # train random forest model
# Step 6 Evaluate model with classification report

# Create me an pipeline follow this step
# Step 1 read raw csv file this file have these column time, x, y, z, label
# Step 2 add column magnitude by compute calculate_magnitude function using columns x, y, z
# Step 3 flatten raw magnitude from step 2 with window size 2 second and overlab 50% and add label base on my old segment_and_flatten_magnitude function
# Step 4 encode label using labelencoder
# Step 5 split train and test (X is raw flattened magnitude and y is label)
# Step 6 Create pipeline with
    # pipeline 1: apply band pass filter into magnitude
    # pipeline 2: compute fft with flattened data and keep first half (+1 for DC component)
    # pipeline 3: train random forest model

# Then export this pipeline and give an example to use this pipeline for predict new raw calculated magnitude

# Follow my step you can see my code below. Then create me an pipeline for combine all these step for easy to deploy and save pipeline with joblip
# and give me example for use that pipeline. Make that pipeline easy to predict like just feed raw magnitude (not filter) and model can predict final 


# Here is my python code


In [ ]:
import numpy as np
import pandas as pd
import joblib
from scipy.signal import butter, lfilter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Function to calculate magnitude
def calculate_magnitude(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

# Function to apply band-pass filter
def bandpass_filter(data, lowcut=0.4, highcut=15, fs=50, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, data)

# Function to segment and flatten magnitude
def segment_and_flatten(df, window_size_sec=2, overlap=0.5, sampling_rate=50):
    df = df.sort_values(by='time').reset_index(drop=True)
    window_samples = int(window_size_sec * sampling_rate)
    step_size = int(window_samples * (1 - overlap))
    flattened_rows = []

    for start_idx in range(0, len(df) - window_samples + 1, step_size):
        window = df.iloc[start_idx : start_idx + window_samples].reset_index(drop=True)
        row_dict = {'start_time': window.loc[0, 'time'], 'label': label}
        for i in range(window_samples):
            row_dict[f'x{i+1}'] = window.loc[i, 'magnitude']
        flattened_rows.append(row_dict)

    return pd.DataFrame(flattened_rows)

# Function to compute FFT
def compute_fft(df, num_samples=100):
    feature_columns = [f'x{i+1}' for i in range(num_samples)]
    fft_features = []
    for _, row in df.iterrows():
        ts = row[feature_columns].values.astype(np.float64)
        fft_vals = np.fft.fft(ts)
        fft_mag = np.abs(fft_vals)
        fft_half = fft_mag[:num_samples // 2 + 1]
        fft_features.append(fft_half)
    fft_df = pd.DataFrame(fft_features, columns=[f'fft_{i+1}' for i in range(num_samples // 2 + 1)])
    return pd.concat([df[['start_time', 'label']], fft_df], axis=1)

# Load and process dataset
def load_and_process_dataset(df):
    df['magnitude'] = calculate_magnitude(df['x'], df['y'], df['z'])
    df['filtered_magnitude'] = bandpass_filter(df['magnitude'])
    segmented_data = segment_and_flatten(df)
    fft_data = compute_fft(segmented_data, num_samples=segmented_data.shape[1] - 2)
    return fft_data

# Train model
def train_pipeline(df):
    df = load_and_process_dataset(df)
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
    y = df['label_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    pipeline = Pipeline([
        ('scaler', scaler),
        ('classifier', model)
    ])
    joblib.dump({'pipeline': pipeline, 'label_encoder': le}, 'activity_recognition_pipeline.pkl')
    print("Pipeline saved successfully.")

# Predict function
def predict_new_data(raw_magnitude):
    data = joblib.load('activity_recognition_pipeline.pkl')
    pipeline = data['pipeline']
    le = data['label_encoder']
    raw_magnitude = np.array(raw_magnitude).reshape(1, -1)
    prediction = pipeline.predict(raw_magnitude)
    return le.inverse_transform(prediction)[0]



#Version 2 from gpt

In [1]:
import numpy as np
import pandas as pd
import joblib
from scipy.signal import butter, lfilter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from preprocess import PreprocessAccel
from sklearn.neighbors import KNeighborsClassifier

pre_acc = PreprocessAccel(sampling_rate=50)

# Function to calculate magnitude
def calculate_magnitude(x, y, z):
    return np.sqrt(x**2 + y**2 + z**2)

# Function to apply band-pass filter
def bandpass_filter(data, lowcut=0.4, highcut=15, fs=50, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, data)

# Function to compute FFT
def compute_fft(data):
    fft_vals = np.fft.fft(data)
    fft_mag = np.abs(fft_vals)
    return fft_mag[: len(fft_mag) // 2 + 1]

# Load and process dataset
def load_and_process_dataset(df):
    df['magnitude'] = calculate_magnitude(df['x'], df['y'], df['z'])
    df['filtered_magnitude'] = bandpass_filter(df['magnitude'])
    return df

# Predict function
def predict_new_data(raw_magnitude):
    data = joblib.load('activity_recognition_pipeline.pkl')
    pipeline = data['pipeline']
    le = data['label_encoder']
    filtered_magnitude = bandpass_filter(raw_magnitude)
    fft_features = compute_fft(filtered_magnitude)
    fft_features = np.array(fft_features).reshape(1, -1)
    prediction = pipeline.predict(fft_features)
    return le.inverse_transform(prediction)[0]

In [5]:
# Train model
def train_pipeline(df):
    # df = load_and_process_dataset(df)

    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
    y = df['label_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    pipeline = Pipeline([
        ('scaler', scaler),
        ('classifier', model)
    ])
    joblib.dump({'pipeline': pipeline, 'label_encoder': le}, 'activity_recognition_pipeline.pkl')
    print("Pipeline saved successfully.")

In [2]:
# Train model
def train_pipeline(df, max_iter=500):
    hidden_layer_sizes=(100, 80, 50, 30, 20, 10) # for data that have filter
    # hidden_layer_sizes=(100, 90, 80, 70, 50, 30, 20, 10) # for data that NOT pass filter
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
    y = df['label_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, 
                          activation='relu', solver='adam', 
                          max_iter=max_iter, random_state=42, verbose=True)
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    
    pipeline = Pipeline([
        ('scaler', scaler),
        ('classifier', model)
    ])
    
    joblib.dump({'pipeline': pipeline, 'label_encoder': le}, 'activity_recognition_pipeline.pkl')
    print("Pipeline saved successfully.")

In [7]:
# Train model
def train_pipeline(df, n_neighbors=5):
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    X = df.drop(['start_time', 'label', 'label_encoded'], axis=1)
    y = df['label_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    
    pipeline = Pipeline([
        ('scaler', scaler),
        ('classifier', model)
    ])
    
    joblib.dump({'pipeline': pipeline, 'label_encoder': le}, 'activity_recognition_pipeline.pkl')
    print("Pipeline saved successfully.")

In [8]:
# Example usage
# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win5_lab50.csv'
# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_no_fft_win5_lab50.csv'
# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\combine_time_feq_win5_lab50.csv'

# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\NO_FILTER_with_fft_win5_lab50.csv'
# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\NO_FILTER_flattened_no_fft_win5_lab50.csv'
# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\NO_FILTER_combine_time_feq_win5_lab50.csv'

file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_with_fft_win2_lab50.csv'
# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\flattened_no_fft_win2_lab50.csv'
# file_path = r'C:\Git-Repository\dsp_project\ignore_dir\datasets\ready_to_train\combine_time_feq_win2_lab50.csv'
train_pipeline(pd.read_csv(file_path))

Classification Report:
                             precision    recall  f1-score   support

            climbing_stairs       0.79      0.86      0.82       170
          descending_stairs       0.91      0.65      0.76       158
                    nothing       0.97      0.93      0.95       148
                    running       1.00      0.97      0.98       145
sitting_standing_transition       0.88      0.99      0.93       146
                    walking       0.81      0.94      0.87       154

                   accuracy                           0.88       921
                  macro avg       0.89      0.89      0.89       921
               weighted avg       0.89      0.88      0.88       921

Pipeline saved successfully.


c:\Users\tanar\anaconda3\envs\final_project_dsp\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\tanar\anaconda3\envs\final_project_dsp\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\tanar\anaconda3\envs\final_project_dsp\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\tanar\anaconda3\envs\final_project_dsp\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\tanar\anaconda3\envs\final_project_dsp\lib\subprocess.py", line 1456, in _execute_

In [29]:
example_magnitude = [-1.9269720474086889,-3.9535963264726286,-3.610971491888949,-1.5433760008604405,-1.0043608153004953,-1.9522487696309168,-1.999174826152136,-1.043848946817768,-0.4439468039369165,-0.4566944243320182,-0.5926976645717903,-0.5908331289349239,-0.4056274775312546,-0.0694776144812227,0.2828222622289001,0.5267270334145863,0.6982578625378797,0.9474634251276348,1.2885017430589882,1.4858023337644424,1.430525202945011,1.4563666249239935,1.8158920460947787,2.2000462505921936,2.12756586082568,1.4975846707645968,0.972249513894111,1.301180820290731,2.0123606308095234,2.162261137596188,1.5793890019468768,0.3827977373727375,-1.2119609816043235,-2.459902067055804,-2.666356906532143,-2.481980196564871,-3.018674754067028,-3.519102542265938,-2.6536894542926195,-1.0332864090202758,-0.070148167838492,-0.1790101227123757,-0.7484022604247628,-0.9467005668292648,-0.8014474142761727,-0.9281826034890698,-1.2366379257751825,-1.0753412035207053,-0.3322344225804168,0.3628425988153955,0.5393957407884269,0.5830972494506156,0.9613680149845492,1.3357289531582752,1.3514832210972518,1.4206131162721622,1.68262701986672,1.5580493159841502,1.2508935629139624,1.660939949364311,2.565976800171682,2.9096783168570948,2.2424480890876985,0.8658109422943936,-0.7987488461969393,-2.486874517578699,-3.5519096491785698,-3.3399223152158948,-2.318179969057168,-1.4094299334456757,-0.9352914011430322,-0.8219382569032276,-0.8991242537729679,-0.9398732494449176,-0.8310274050188793,-0.7228645106825701,-0.7510581406447109,-0.7381344693296573,-0.4930819479945015,-0.1542708295096546,0.0284438009471157,0.1220863638406615,0.3657008699281848,0.6690254623208504,0.9497325183856876,1.4821707621709024,2.141504495114866,2.474538318291468,2.3603135965983424,1.6444185724062756,0.7777831799208994,0.9755491479323568,1.975839468590048,2.4482037062147004,2.3999483239327843,2.0909369476494297,0.8924821288379312,-0.978097045438806,-2.29987317233632,-2.5981731046374943,-2.210980446452718,-1.5691300326582116,-1.3473083049818868,-1.95944097144001,-2.6775893447021084,-2.6945580052223823,-2.401199596941323,-2.2588896994004517,-2.0337073114692195,-1.646284320723402,-1.1032705519467392,-0.3587025724412591,0.3244097461288157,0.7678285201254511,0.9848272918582684,1.0794026186193304,1.3327719237696032,1.6817339757651468,1.877161536782004,2.1249046552208792,2.461519670047495,2.533813341497132,2.314977790147115,2.0017182939030835,1.8916567860968063,2.2341138850389104,2.565135576455762,2.293103710547705,1.4933040255747392,0.1923453775197899,-1.640425977970132,-3.205761189714095,-3.610038644520169,-2.9646300917863497,-2.0731264280508803,-1.650244066315334,-1.6610305559776517,-1.618938444994348,-1.469779227659456,-1.4419272549603843,-1.5711910928288049,-1.6795934280613485,-1.5603054033236847,-1.2277690209581662,-0.7993304404616814,-0.4029360048775012,-0.1634029563760262,0.1693751551831318,0.8427253647422082,1.4942101728972284,2.0229793904947444,2.9424304289826506,4.000556960810066,4.204956740724863,3.4243847436159247,2.3268418022257262,1.3989586795819091,0.9966145197092076,1.291708419625052,1.700899119757892,1.2830036070716635,0.1230212562593542,-0.9581157808738018,-1.9596140727414189,-2.944568692854584,-3.2671521863682083,-3.03437501921387,-2.889143474906973,-2.699603575813858,-2.3046555019983805,-1.9733610462578863,-1.701956256748684,-1.41261635328417,-1.194631095024203,-0.9610527270931296,-0.7138555537372812,-0.6456777413138093,-0.6505699239485324,-0.4943131880800978,-0.109508566695326,0.5848939502577961,1.2976389604814025,1.3380442678598516,0.9402296394941344,1.12257844334081,1.8098644195539917,2.109203942485512,1.9864754172332453,1.977111793048882,2.2240786652146065,2.6871984421202653,3.176933545296387,3.088378073188342,2.246547249667172,1.379274339782507,0.6521979988197503,-0.7563041784186838,-2.6900940609522843,-3.650825592494744,-2.909330093941332,-1.65271075805177,-1.4811717152950346,-2.055958273304232,-1.8728267415950144,-0.97254591691293,-0.524532178761456,-0.776968942682341,-1.0306447280422102,-0.7836006827310475,-0.3900892912000824,-0.4425135542328332,-0.6044816509149412,-0.2188095043825917,0.3205474840284611,0.4355232604877098,0.5108945148963782,0.9613020527912634,1.528395904288081,1.8291086350931507,1.871443535082636,1.9419194074282489,1.924634176566096,1.4926010207270948,0.9509438606580184,0.7442478227815879,0.8658160358410039,1.223625553659427,1.7525586361993717,2.2609100170923675,1.9214408622428472,-0.0474500142030707,-2.216141662567205,-2.623113859181798,-2.145548034692475,-2.322759725969924,-2.677601733998462,-2.614068875721906,-2.363115665113501,-1.8493477371264655,-1.2320038606783066,-1.1707388533367595,-1.453275804407009,-1.3109130437147578,-0.8668969470535376,-0.613003649769417,-0.432636088535272,-0.0978053981660688,0.3139758029769334,0.7174644270761752,1.0379724676543065]
print("Predicted activity:", predict_new_data(example_magnitude))


Predicted activity: climbing_stairs


d:\anaconda\envs\final_project_dsp\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [11]:

root_path = r"D:\git_repository\dsp_project\ignore_dir\datasets\cleaned_data_many_position"

# Function to extract label from filename
def extract_label(filename):
    match = re.match(r'([a-zA-Z]+)', filename)
    return match.group(1) if match else filename

# Process all CSV files in the directory and subdirectories
all_data = []
for dirpath, _, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith(".csv"):  # Process only CSV files
            file_path = os.path.join(dirpath, filename)
            label = extract_label(filename)
            data = pd.read_csv(file_path)
            # print(data.sample())
            data['label'] = label
            all_data.append(data.drop(['filtered_magnitude', 'magnitude'], axis=1))

# Combine all processed data
if all_data:
    merged_df = pd.concat(all_data, ignore_index=True)
    print("Merged DataFrame shape:", merged_df.shape)
else:
    print("No CSV files found.")

train_pipeline(merged_df)

Merged DataFrame shape: (300420, 5)
Classification Report:
              precision    recall  f1-score   support

      D:\git       1.00      1.00      1.00        43

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43

Pipeline saved successfully.


In [12]:
merged_df

,time,x,y,z,label
0,24181,-1.86,9.04,-1.62,climbing
1,24208,-1.33,9.60,-2.19,climbing
2,24235,-1.32,10.29,-2.32,climbing
3,24261,-1.81,10.24,-2.96,climbing
4,24283,-1.85,9.80,-3.04,climbing
...,...,...,...,...,...
300415,269668,-2.74,8.79,-0.99,walking
300416,269689,-2.57,8.96,-1.08,walking
300417,269712,-2.03,9.20,-1.05,walking
300418,269735,-2.54,9.42,-1.11,walking
